In [4]:
from pyscf import gto,scf
import pyscf
import matplotlib.pyplot as plt
import basis_set_exchange as bse

from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [5]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',
                      basis=bse.get_basis("pcX-3",fmt="nwchem",elements=[5,6,7,8,9]))
mf=scf.RHF(CO)
CO.nao

168

In [6]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-3",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-3",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-3",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-3",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [7]:
optimizer2at("C","O")

(2.0821799999999997, -112.7924907112811)

In [8]:
optimizer2at("N","N")

(2.01335, -108.99635414735519)

In [9]:
optimizer2at("B","F")

(2.35193, -124.16889487546402)

In [10]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

(2.0822399999999996, -112.79228687491637)

In [11]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

(2.0823, -112.79239330808514)

In [12]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

(2.35201, -124.16873379958503)

In [13]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])

(2.01342, -108.99633460349818)

In [14]:
import pandas as pd

In [16]:
q=[]
l={"basis set":"pcX-3"
    ,"nAO":168
    ,"COm":2.0821799999999997
    ,"COe":-112.7924907112811
    ,"NNm":2.01335
    ,"NNe":-108.99635414735519
    ,"BFm":2.35193 
    ,"BFe":-124.16889487546402
    ,"CO@NNm":2.0822399999999996
    ,"CO@NNe": -112.79228687491637
    ,"CO@BFm":2.0823
    ,"CO@BFe":-112.79239330808514
    ,"BF@COm":2.35201
    ,"BF@COe":-124.16873379958503
    ,"NN@COm":2.01342
    ,"NN@COe":-108.99633460349818
}
q.append(l)

In [18]:
df=pd.DataFrame(q)

In [22]:
(pd.read_pickle("pcX2").append(df)).to_pickle("pcX23")